# Análisis de los Datos Abiertos de la Ciudad de México

In [38]:
import requests
import pandas as pd
import numpy as np

### Funciones útiles

In [39]:
def api_requests_response(url):
    """Realiza el request y devuelve un DataFrame

    Args:
        url (string): url de la api para extraer información

    Returns:
        DataFrame: devuelve un dataframe con la información
    """
    response = requests.get(str(url))
    data = response.json()
    return pd.DataFrame(data['result']['records'])

def distancias(x, y, alcaldia, indice):
    if indice <16:
        dist.append((x-alcaldia.position_latitude[indice])**2 + (y - alcaldia.position_longitude[indice])**2)
        distancias(x, y, alcaldia, indice +1)

def unidades_dispon():
    """Funcion de devuelve una lista del id ("id") de las unidades disponibles,
    de acuerdo al campo "vehicle_currenten_status": 1 = Activa, 2 = Inactiva

    Returns:
        list: lista de unidades activas
    """
    unid_disp = ubicacion_metrobus[ubicacion_metrobus['vehicle_current_status']==1]
    return list(unid_disp['id'])

def ubic_por_ID(id):
    """Función que devuelve el nombre de la alcaldia según el id ("id") ingresado

    Args:
        id (int): Argumento que corresponde al campo "id"

    Returns:
        str: nombre de la alcaldía
    """
    return ubicacion_metrobus[ubicacion_metrobus['id']==id]['nomgeo']

def alcaldias_dispon():
    """Función que devuelve una lista de las alcaldías
    en las que se encuentra al menos una ubicación del metrobus

    Returns:
        list: lista de los nombres de las alcaldías (campo "nomgeo")
    """
    return list(ubicacion_metrobus['nomgeo'].unique())

def unidades_por_alcaldia(alcaldia_id):
    """_summary_

    Args:
        alcaldia_id (_type_): _description_

    Returns:
        _type_: _description_
    """
    ubicacion_metrobus_por_alcaldia = ubicacion_metrobus[ubicacion_metrobus['alcaldia_id']==alcaldia_id]
    return ubicacion_metrobus_por_alcaldia['id'].count()


Las siguientes URL's contienen la información de las alcaldías y las ubicaciones del metrobus

In [40]:
# url de las api's
api_alcaldia_url = 'https://datos.cdmx.gob.mx/api/3/action/datastore_search?resource_id=e4a9b05f-c480-45fb-a62c-6d4e39c5180e&limit=16'
api_ubicacion_unidades_metrobus = 'https://datos.cdmx.gob.mx/api/3/action/datastore_search?resource_id=ad360a0e-b42f-482c-af12-1fd72140032e&limit=207'


In [41]:
#Llamada a la función para obtener la información en dataframes
alcaldias = api_requests_response(api_alcaldia_url)
ubicacion_metrobus = api_requests_response(api_ubicacion_unidades_metrobus)


In [42]:
#Se ordenan de forma ascendente los DataFrames "alcaldias" y "ubicacion_metrobus", mdeiante los campos "id" en ambos
alcaldias = alcaldias.sort_values('id')
ubicacion_metrobus = ubicacion_metrobus.sort_values('id')

Sin embargo, no hay relación explícita entre ambos dataframes, se opta por realizar el siguiente procedimiento

In [43]:

#Lista de las columnas a las que se les cambiará el tipo de dato
positions=['position_latitude', 'position_longitude']

#Cambio de tipo de dato de str a float
alcaldias[positions] = alcaldias.geo_point_2d.str.split(',', expand=True)
alcaldias[positions] = alcaldias[positions].astype('float64')

In [44]:
#Loop para rellenar una lista que contiene el nombre de la alcaldia
#considerando la distancia mínima de la ubicacion de metrobus con la alcaldia
nomgeo_list = []
alcaldia_id_list =[]
alcaldia_cve_mun = []
for lab, row in ubicacion_metrobus.iterrows():
    dist = []
    distancias(row.position_latitude, row.position_longitude,alcaldias,0)
    nomgeo_list.append(alcaldias.iloc[dist.index(min(dist)), 2])
    alcaldia_id_list.append(alcaldias.iloc[dist.index(min(dist)), 1])
    alcaldia_cve_mun.append(alcaldias.iloc[dist.index(min(dist)), 3])

In [45]:
#Se agregan nuevas columnas "nomgeo", "alcaldia_id" y "cve_num" de acuerdo a cada ubicación
ubicacion_metrobus = ubicacion_metrobus.assign(nomgeo = nomgeo_list)
ubicacion_metrobus = ubicacion_metrobus.assign(alcaldia_id = alcaldia_id_list)
ubicacion_metrobus = ubicacion_metrobus.assign(cve_num = alcaldia_cve_mun)

In [46]:
#Muestra la lista de las unidades activas
print(f'La siguiente es la lista de las unidades activas \n {unidades_dispon()} \n')

#Muestra la alcaldia correspondiente al "id" de ubicacion_metrobus
id_ubicacion = int(input("Ingrese el id de la ubicacion: "))
print(f'La ubicación de metrobús con id={id_ubicacion} pertenece a la alcaldía {ubic_por_ID(id_ubicacion)} \n')

#Muestra una lista de los nombres en los que se encuentra al menos una ubicación del metrobus
print(f'La siguiente es una lista de las alcaldias con al menos una ubicación de metrobús \n {alcaldias_dispon()} \n')

#Muestra un conteo de las unididades por alcaldia de acuerdo al "id" de la alcaldia
id_alcaldia = int(input('Ingrese el id de la alcaldia: '))
print(f'La alcaldía con id = {id_alcaldia} tiene el siguiente número de uicaciones de metrobús {unidades_por_alcaldia(id_alcaldia)} \n')

La siguiente es la lista de las unidades activas [1, 3, 6, 8, 20, 21, 22, 23, 25, 28, 36, 45, 47, 49, 50, 51, 55, 57, 66, 67, 72, 74, 83, 84, 85, 87, 88, 92, 94, 98, 99, 102, 112, 113, 114, 116, 118, 119, 120, 121, 130, 144, 151, 153, 156, 157, 160, 162, 163, 164, 166, 168, 174, 178, 181, 182, 183, 184, 194, 196] 

La alcaldia de la ubicacion de metrobús con id=7 es: 7    Benito Jurez
Name: nomgeo, dtype: object 

La siguiente es la lista de las alcaldias con al menos una ubicacion de metrobús ['Coyoacn', 'Venustiano Carranza', 'Iztacalco', 'Iztapalapa', 'Benito Jurez', 'Cuauhtmoc', 'Azcapotzalco', 'Gustavo A. Madero', 'Miguel Hidalgo', 'Milpa Alta'] 

El número total de ubicaciones de metrobús de la alcaldía con id = 8 es: 18 

